# Performance

In [ ]:
import numpy as np
import sqlite3 as sq
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
import requests
from mpl_toolkits.basemap import Basemap
import time
import json
import tldextract

%matplotlib inline

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

plt.rcParams.update(params)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
def buildExtractedDomain(extracted):
    result = ""
    if extracted.subdomain != "" and extracted.subdomain != "*":
        result += extracted.subdomain + "."
    if extracted.domain != "" and extracted.suffix != "":
        result += extracted.domain + "." + extracted.suffix
    return result

def mapToCommonName(info):
    commonName = buildExtractedDomain(tldextract.extract(info["CommonName"]))
    if commonName != "":
        return commonName
    if info["DNSNames"] == None:
        return None
    return buildExtractedDomain(tldextract.extract(info["DNSNames"][0]))

In [ ]:
connection_misc = sq.connect("single.query.response.times/misc.db")
connection_misc.row_factory = sq.Row

# ---------------------------------------------------------------------------------

# Data preparation

In [ ]:
#load dns_measurements
measurements = pd.read_parquet('single.query.response.times/single.query.response.times.parquet.gzip')

In [ ]:
print("all measurements: " + str(len(measurements)))
print('doq: ' + str(len(measurements[measurements['protocol'] == 'quic'])))
print('doh: ' + str(len(measurements[measurements['protocol'] == 'https'])))
print('dot: ' + str(len(measurements[measurements['protocol'] == 'tls'])))
print('dotcp: ' + str(len(measurements[measurements['protocol'] == 'tcp'])))
print('doudp: ' + str(len(measurements[measurements['protocol'] == 'udp'])))


In [ ]:
print("measurements per vp: ")
print('AF-SOUTH-1: ' + str(len(measurements[measurements['vp'] == 'AF-SOUTH-1'])))
print('AP-NORTHEAST-3: ' + str(len(measurements[measurements['vp'] == 'AP-NORTHEAST-3'])))
print('AP-SOUTHEAST-3: ' + str(len(measurements[measurements['vp'] == 'AP-SOUTHEAST-3'])))
print('EU-CENTRAL-1: ' + str(len(measurements[measurements['vp'] == 'EU-CENTRAL-1'])))
print('SA-EAST-1: ' + str(len(measurements[measurements['vp'] == 'SA-EAST-1'])))
print('US-WEST-1: ' + str(len(measurements[measurements['vp'] == 'US-WEST-1'])))

### All measurements of Resolvers which answered at least once with an r_code for every protocol

In [ ]:
# number of resolvers per protocol

measuremens_ips_quic_responsive = set(measurements[~(measurements['r_code'].isna()) & (measurements['protocol'] == 'quic')]['ip'].unique())
measuremens_ips_https_responsive = set(measurements[~(measurements['r_code'].isna()) & (measurements['protocol'] == 'https')]['ip'].unique())
measuremens_ips_tls_responsive = set(measurements[~(measurements['r_code'].isna()) & (measurements['protocol'] == 'tls')]['ip'].unique())
measuremens_ips_tcp_responsive = set(measurements[~(measurements['r_code'].isna()) & (measurements['protocol'] == 'tcp')]['ip'].unique())
measuremens_ips_udp_responsive = set(measurements[~(measurements['r_code'].isna()) & (measurements['protocol'] == 'udp')]['ip'].unique())

print('doq: ' + str(len(measuremens_ips_quic_responsive)))
print('doh: ' + str(len(measuremens_ips_https_responsive)))
print('dot: ' + str(len(measuremens_ips_tls_responsive)))
print('dotcp: ' + str(len(measuremens_ips_tcp_responsive)))
print('doudp: ' + str(len(measuremens_ips_udp_responsive)))


In [ ]:
# number of resolvers supporting every protocol
measuremens_ips_all_responsive = set.intersection(measuremens_ips_quic_responsive, measuremens_ips_https_responsive, measuremens_ips_tls_responsive, measuremens_ips_tcp_responsive, measuremens_ips_udp_responsive)
len(measuremens_ips_all_responsive)


In [ ]:
# number of resolvers supporting every protocol but dotcp
len(set.intersection(measuremens_ips_quic_responsive, measuremens_ips_https_responsive, measuremens_ips_tls_responsive, measuremens_ips_udp_responsive))


In [ ]:
measurements_responsive = measurements[measurements['ip'].isin(measuremens_ips_all_responsive)]

In [ ]:
print("all measurements: " + str(len(measurements_responsive)))
print('doq: ' + str(len(measurements_responsive[measurements_responsive['protocol'] == 'quic'])))
print('doh: ' + str(len(measurements_responsive[measurements_responsive['protocol'] == 'https'])))
print('dot: ' + str(len(measurements_responsive[measurements_responsive['protocol'] == 'tls'])))
print('dotcp: ' + str(len(measurements_responsive[measurements_responsive['protocol'] == 'tcp'])))
print('doudp: ' + str(len(measurements_responsive[measurements_responsive['protocol'] == 'udp'])))


# ---------------------------------------------------------------------------------

# Meta

## Stats

In [ ]:
# total
measurements_responsive_per_protocol = measurements_responsive.groupby('protocol').size().reset_index(name = "total")

mask = (~measurements_responsive['r_code'].isna()) & (measurements_responsive['error'].isnull()) & (~measurements_responsive['total_time'].isna()) & (measurements_responsive['total_time'] < 5e9)

# successful
measurements_successful = measurements_responsive[mask]
measurements_successful_per_protocol = measurements_successful.groupby('protocol').size().reset_index(name = "sucessful")

measuremens_ips_quic_successful = set(measurements_successful[measurements_successful['protocol'] == 'quic']['ip'].unique())
measuremens_ips_https_successful = set(measurements_successful[measurements_successful['protocol'] == 'https']['ip'].unique())
measuremens_ips_tls_successful = set(measurements_successful[measurements_successful['protocol'] == 'tls']['ip'].unique())
measuremens_ips_tcp_successful = set(measurements_successful[measurements_successful['protocol'] == 'tcp']['ip'].unique())
measuremens_ips_udp_successful = set(measurements_successful[measurements_successful['protocol'] == 'udp']['ip'].unique())

measuremens_ips_all_successful = set.intersection(measuremens_ips_quic_successful, measuremens_ips_https_successful, measuremens_ips_tls_successful, measuremens_ips_tcp_successful, measuremens_ips_udp_successful)
print(len(measuremens_ips_all_successful))
# 309 resolvers are successful


# failed
measurements_failed = measurements_responsive[~mask]
measurements_failed_per_protocol = measurements_failed.groupby('protocol').size().reset_index(name = "failed")


In [ ]:
measurements_stats = measurements_responsive_per_protocol.merge(measurements_successful_per_protocol)
measurements_stats = measurements_stats.merge(measurements_failed_per_protocol)
measurements_stats['failed_rel'] = (measurements_stats['failed'] / measurements_stats['total']).map(" {:.1%}".format) 
measurements_stats


## Load Common Names

In [ ]:
certs = pd.read_csv("single.query.response.times/certificates.csv", converters={"info":json.loads}, header = None, names = ["ip", "protocol", "port", "info"])
quic_certs_last_week = certs[certs["protocol"] == "quic"]
quic_certs_last_week["common_name"] = quic_certs_last_week["info"].apply(mapToCommonName)

In [ ]:
quic_certs_last_week = quic_certs_last_week.drop_duplicates(subset = ['ip', 'common_name'])

In [ ]:
count_by_common_name = quic_certs_last_week.groupby("common_name").size().reset_index(name = "count").set_index("common_name")
count_by_common_name["rel"] = (count_by_common_name["count"] / count_by_common_name["count"].sum()) * 100

# ---------------------------------------------------------------------------------

# Regional Distribution

In [ ]:
# get regional data from ip addresses
data = {}
for row in measuremens_ips_all_responsive:
    #print(row)
    r = requests.get("http://ip-api.com/json/" + row)
    json = r.json()
    data[row] = {
        "country": json["country"],
        "countryCode": json["countryCode"],
        "region": json["region"],
        "regionName": json["regionName"],
        "city": json["city"],
        "zip": json["zip"],
        "lat": json["lat"],
        "lon": json["lon"],
        "org": json["org"],
        "as": json["as"],
        "ip": row
    }
    time.sleep(1.5)
measuremens_ips_all_responsive_regional = pd.DataFrame.from_dict(data, orient = "index")

In [ ]:
measuremens_ips_all_responsive_regional

## By Continent

In [ ]:
countriesContinents = pd.read_csv("single.query.response.times/Countries-Continents.csv", index_col = False)
renamed = countriesContinents.columns.tolist()
renamed[0] = "country"
countriesContinents.columns = renamed

def mapToContinent(row):
    if row["region"] == "Americas" and row["intermediate-region"] == "South America":
        return "South America (SA)"
    elif row["region"] == "Americas":
        return "North America (NA)"
    elif row["region"] == "Asia":
        return "Asia (AS)"
    elif row["region"] == "Europe":
        return "Europe (EU)"
    elif row["region"] == "Oceania":
        return "Oceania (OC)"
    elif row["region"] == "Africa":
        return "Africa (AF)"
    return row["region"]

countriesContinents["continent"] = countriesContinents.apply(mapToContinent, axis = 1)

In [ ]:
measuremens_ips_all_responsive_regional = pd.merge(measuremens_ips_all_responsive_regional, countriesContinents, on = "country")
measuremens_ips_all_responsive_regional['continent'].value_counts()


In [ ]:
dox_by_continent = pd.concat([measuremens_ips_all_responsive_regional['continent'].value_counts(),
                                     measuremens_ips_all_responsive_regional['continent'].value_counts(normalize=True)], axis=1)
dox_by_continent = dox_by_continent.reset_index() 
dox_by_continent.columns = ['Continent', 'abs', 'rel']
dox_by_continent['rel'] = dox_by_continent['rel'].map(" ({:.2%})".format)
dox_by_continent['DoX verified'] = dox_by_continent['abs'].astype(str) + dox_by_continent['rel']
dox_by_continent = dox_by_continent[['Continent', 'DoX verified']]
dox_by_continent

## World map of resolvers

In [ ]:
vantage_points = {
                  'Asia Pacific Northeast':   {'short': 'ap-northeast_', 'location': {'lat': 34.6937, 'lng': 135.5022}},
                  'Africa South':             {'short': 'af-south_', 'location': {'lat':  -33.9258, 'lng': 18.4232}},
                  'Europe Central':           {'short': 'eu-central_', 'location': {'lat': 50.1155, 'lng': 8.6842}},
                  'Asia Pacific Southeast':   {'short': 'ap-southeast_', 'location': {'lat': -33.8679, 'lng': 151.2073}},
                  'US West':                  {'short': 'us-west_', 'location': {'lat':  37.7749, 'lng': -122.4194}},
                  'South America East':       {'short': 'sa-east_', 'location': {'lat': -23.5475, 'lng': -46.6361}}
                 }

plt.figure(figsize=(12,4))
worldMap = Basemap(lon_0=0, resolution='l')
worldMap.drawcountries(color='#ffffff', linewidth=0.5)
worldMap.fillcontinents(color='#c0c0c0', lake_color='#ffffff')
x, y = worldMap(measuremens_ips_all_responsive_regional["lon"].tolist(), measuremens_ips_all_responsive_regional["lat"].tolist())
plt.plot(x, y, 'ro', markersize = 3)

vpx, vpy = worldMap([8.6842], [50.1155])
plt.plot(vpx, vpy, 'bo', markersize = 5)
vpx, vpy = worldMap([135.5022], [34.6937])
plt.plot(vpx, vpy, 'bo', markersize = 5)
vpx, vpy = worldMap([18.4232], [-33.9258])
plt.plot(vpx, vpy, 'bo', markersize = 5)
vpx, vpy = worldMap([152.2073], [-30.8679]) #([151.2073], [-33.8679])
plt.plot(vpx, vpy, 'bo', markersize = 5)
vpx, vpy = worldMap([-122.4194], [37.7749])
plt.plot(vpx, vpy, 'bo', markersize = 5)
vpx, vpy = worldMap([-44.6361], [-21.5475]) #([-46.6361], [-23.5475])
plt.plot(vpx, vpy, 'bo', markersize = 5)

plt.box(False)
plt.savefig("resolvers-vps-map.pdf", bbox_inches='tight')
plt.show()


## By ASN

In [ ]:
top_n = 10

target_dist_by_asn = pd.concat([measuremens_ips_all_responsive_regional['as'].value_counts().head(top_n),
                                     measuremens_ips_all_responsive_regional['as'].value_counts(normalize=True).head(top_n)], axis=1)
target_dist_by_asn = target_dist_by_asn.reset_index() 
target_dist_by_asn.columns = ['ASN', 'abs', 'rel']
target_dist_by_asn['rel'] = target_dist_by_asn['rel'].map(" ({:.2%})".format)
target_dist_by_asn_table = target_dist_by_asn

target_dist_by_asn_table['ASN'] = target_dist_by_asn_table['ASN'].str.split(' ', expand=True)[0]
target_dist_by_asn_table['ASN'] = target_dist_by_asn_table['ASN'].str.slice(2)
target_dist_by_asn_table

In [ ]:
len(measuremens_ips_all_responsive_regional['as'].value_counts())

# all ASes observed for DoX verified

# ---------------------------------------------------------------------------------

# TCP

## TFO support

In [ ]:
tcp_tfo = pd.read_sql_query("SELECT * FROM fast_open_supports", connection_misc)

In [ ]:
tcp_tfo_support = tcp_tfo[tcp_tfo['support'] != 0]
tcp_tfo_support_ips = set(tcp_tfo_support['ip'].unique())
len(tcp_tfo_support_ips)

#194 resolvers with support for tfo

In [ ]:
tcp_tfo_support_ips_successful = set(measurements_successful['ip'].unique()).intersection(tcp_tfo_support_ips)

len(tcp_tfo_support_ips_successful)
#0 resolver with successful measurements supports tfo

In [ ]:
measuremens_ips_all_responsive_regional[measuremens_ips_all_responsive_regional['ip'].isin(tcp_tfo_support_ips_successful)]


In [ ]:
quic_certs_last_week[quic_certs_last_week['ip'].isin(tcp_tfo_support_ips_successful)]


In [ ]:
#0 resolver with successful measurements which supports tfo

## edns-tcp-keepalive support

In [ ]:
tcp_edns0_keepalive = pd.read_sql_query("SELECT * FROM e_dns0", connection_misc)

In [ ]:
tcp_edns0_keepalive_support = tcp_edns0_keepalive[tcp_edns0_keepalive['support'] != 0]
tcp_edns0_keepalive_support_ips = set(tcp_edns0_keepalive_support['ip'].unique())

len(tcp_edns0_keepalive_support_ips)
#0 resolvers with support for edns0-tcp-keepalive

In [ ]:
tcp_edns0_keepalive_support_ips_successful = set(measurements_successful['ip'].unique()).intersection(tcp_edns0_keepalive_support_ips)
len(tcp_edns0_keepalive_support_ips_successful)

#0 resolver with successful measurements support edns0-tcp-keepalive

In [ ]:
len(quic_certs_last_week[quic_certs_last_week['ip'].isin(tcp_edns0_keepalive_support_ips_successful)])

#0 resolvers with successful measurements which support edns0-tcp-keepalive are all adguard

In [ ]:
set(tcp_edns0_keepalive_support['timeout'] != 0)

#all resolvers have a timeout value of 0, which signals to close the connection after having received the response.

# ---------------------------------------------------------------------------------

# DoQ

In [ ]:
doq_successful = measurements_successful[measurements_successful['protocol'] == 'quic']

### versions

In [ ]:
doq_successful_versions = doq_successful[['q_ui_c_version']]
doq_successful_versions_grouped = doq_successful_versions.groupby(['q_ui_c_version']).size().reset_index(name = "total")
doq_successful_versions_grouped['q_ui_c_version'] = doq_successful_versions_grouped['q_ui_c_version'].astype('int64')
doq_successful_versions_grouped


In [ ]:
print(doq_successful_versions_grouped['total'][0] / doq_successful_versions_grouped['total'].sum())
print(doq_successful_versions_grouped['total'][1] / doq_successful_versions_grouped['total'].sum())
print(doq_successful_versions_grouped['total'][2] / doq_successful_versions_grouped['total'].sum())
print(doq_successful_versions_grouped['total'][3] / doq_successful_versions_grouped['total'].sum())

In [ ]:
doq_successful_doq = doq_successful[['q_ui_c_negotiated_protocol']]

doq_successful_doq_grouped = doq_successful_doq.groupby(['q_ui_c_negotiated_protocol']).size().reset_index(name = "total")
doq_successful_doq_grouped


In [ ]:
print(doq_successful_doq_grouped['total'][0] / doq_successful_doq_grouped['total'].sum())
print(doq_successful_doq_grouped['total'][1] / doq_successful_doq_grouped['total'].sum())
print(doq_successful_doq_grouped['total'][2] / doq_successful_doq_grouped['total'].sum())

In [ ]:
quic_versions = pd.read_sql_query("SELECT * FROM q_versions", connection_misc)
quic_versions['draft_version'].unique()

# only doq draft versions 3,2, and 0 are found


### 0-RTT support

In [ ]:
quic_0_rtt = pd.read_sql_query("SELECT * FROM q0_rtt_supports", connection_misc)

In [ ]:
quic_0_rtt[quic_0_rtt['indicated0_rtt'] != 0]

# 1 resolver indicates 0-RTT support

In [ ]:
quic_0_rtt[quic_0_rtt['used0_rtt'] != 0]

# 1 resolver successfull 0-RTT

In [ ]:
doq_successful[doq_successful['q_ui_c_used0_rtt'] != 0]

# 0-RTT resolvers are not in analysed measurements

### Session resumption

In [ ]:
quic_sr = pd.read_sql_query("SELECT * FROM q0_rtt_supports", connection_misc)

In [ ]:
quic_sr[quic_sr['sends_session_ticket'] != 1]

# All resolvers send session ticket

In [ ]:
quic_sr.groupby('ticket_lifetime').size().reset_index(name = "total")


In [ ]:
measurements_successful_quic_ip_port = measurements_successful[measurements_successful['protocol'] == 'quic'][['ip', 'port']]

measurements_successful_quic_ip = pd.DataFrame()
measurements_successful_quic_ip['ip'] = measurements_successful_quic_ip_port['ip'].unique()
measurements_successful_quic_ip_merged = measurements_successful_quic_ip.merge(quic_sr, left_on=['ip'], right_on=['ip'])
measurements_successful_quic_ip_merged.groupby('ticket_lifetime').size().reset_index(name = "total")


# ---------------------------------------------------------------------------------

# DoH

In [ ]:
doh_successful = measurements_successful[measurements_successful['protocol'] == 'https']

### TLS versions

In [ ]:
doh_successful_tls = doh_successful.groupby('tls_version').size().reset_index(name = "total")
doh_successful_tls


In [ ]:
doh_successful_tls['total'][1] / doh_successful_tls['total'].sum()


### HTTP versions

In [ ]:
doh_successful_http = doh_successful.groupby('http_version').size().reset_index(name = "total")
doh_successful_http


In [ ]:
doh_successful_http['total'][1] / doh_successful_http['total'].sum()

# ---------------------------------------------------------------------------------

# DoT

In [ ]:
dot_successful = measurements_successful[measurements_successful['protocol'] == 'tls']

### TLS versions

In [ ]:
dot_successful_tls = dot_successful.groupby('tls_version').size().reset_index(name = "total")
dot_successful_tls


In [ ]:
dot_successful_tls['total'][1] / dot_successful_tls['total'].sum()


# ---------------------------------------------------------------------------------

# DoTCP

In [ ]:
dotcp_successful = measurements_successful[measurements_successful['protocol'] == 'tcp']

# ---------------------------------------------------------------------------------

# DoUDP

In [ ]:
doudp_successful = measurements_successful[measurements_successful['protocol'] == 'udp']

# ---------------------------------------------------------------------------------

# Protocol Comparison

## Resolve Times

In [ ]:
doq_query_time = list(doq_successful['query_time'] / 1000000)
doh_query_time = list(doh_successful['query_time'] / 1000000)
dot_query_time = list(dot_successful['query_time'] / 1000000)
dotcp_query_time = list(dotcp_successful['query_time'] / 1000000)
doudp_query_time = list(doudp_successful['query_time'] / 1000000)

In [ ]:
print('Samples:')
print(len(doq_query_time))
print(len(doh_query_time))
print(len(dot_query_time))
print(len(dotcp_query_time))
print(len(doudp_query_time))

In [ ]:
print('Means:')
print(np.mean(doq_query_time))
print(np.mean(doh_query_time))
print(np.mean(dot_query_time))
print(np.mean(dotcp_query_time))
print(np.mean(doudp_query_time))

print('Rate DoQ to DoUDP mean: ' + str(np.mean(doq_query_time) / np.mean(doudp_query_time)))
print('Rate DoQ to DoH mean: ' + str(np.mean(doq_query_time) / np.mean(doh_query_time)))

In [ ]:
print('Medians:')
print(np.median(doq_query_time))
print(np.median(doh_query_time))
print(np.median(dot_query_time))
print(np.median(dotcp_query_time))
print(np.median(doudp_query_time))

print('Rate DoQ to DoUDP median: ' + str(np.median(doudp_query_time) / np.median(doq_query_time)))
print('Rate DoQ to DoTCP median: ' + str(np.median(dotcp_query_time) / np.median(doq_query_time)))

## Handshake time

In [ ]:
doq_successful['handshake_time'] = doq_successful['q_ui_c_handshake_duration']
doh_successful['handshake_time'] = doh_successful['tcp_handshake_duration'] + doh_successful['tls_handshake_duration']
dot_successful['handshake_time'] = dot_successful['tcp_handshake_duration'] + dot_successful['tls_handshake_duration']
dotcp_successful['handshake_time'] = dotcp_successful['tcp_handshake_duration']
doudp_successful['handshake_time'] = np.NaN


In [ ]:
doq_handshake_time = list(doq_successful['handshake_time'] / 1000000)
doh_handshake_time = list(doh_successful['handshake_time'] / 1000000)
dot_handshake_time = list(dot_successful['handshake_time'] / 1000000)
dotcp_handshake_time = list(dotcp_successful['handshake_time'] / 1000000)

In [ ]:
print('Samples:')
print(len(doq_handshake_time))
print(len(doh_handshake_time))
print(len(dot_handshake_time))
print(len(dotcp_handshake_time))


In [ ]:
print('Means:')
print(np.mean(doq_handshake_time))
print(np.mean(doh_handshake_time))
print(np.mean(dot_handshake_time))
print(np.mean(dotcp_handshake_time))

In [ ]:
print('Medians:')
print(np.median(doq_handshake_time))
print(np.median(doh_handshake_time))
print(np.median(dot_handshake_time))
print(np.median(dotcp_handshake_time))

# Heatmap Plots

In [ ]:
#create new df with metrics relevant for heatmap plots
measurement_successful_heatmap = doq_successful[['protocol', 'query_time', 'handshake_time', 'vp']]
measurement_successful_heatmap = pd.concat([measurement_successful_heatmap, doh_successful[['protocol', 'query_time', 'handshake_time', 'vp']]])
measurement_successful_heatmap = pd.concat([measurement_successful_heatmap, dot_successful[['protocol', 'query_time', 'handshake_time', 'vp']]])
measurement_successful_heatmap = pd.concat([measurement_successful_heatmap, dotcp_successful[['protocol', 'query_time', 'handshake_time', 'vp']]])
measurement_successful_heatmap = pd.concat([measurement_successful_heatmap, doudp_successful[['protocol', 'query_time', 'handshake_time', 'vp']]])
measurement_successful_heatmap['hsr'] = measurement_successful_heatmap['handshake_time'] / measurement_successful_heatmap['query_time']
measurement_successful_heatmap['total'] = measurement_successful_heatmap['handshake_time'].fillna(0) + measurement_successful_heatmap['query_time']

In [ ]:
#change captions
measurement_successful_heatmap.loc[measurement_successful_heatmap.protocol == 'quic', 'protocol'] = 'DoQ'
measurement_successful_heatmap.loc[measurement_successful_heatmap.protocol == 'https', 'protocol'] = 'DoH'
measurement_successful_heatmap.loc[measurement_successful_heatmap.protocol == 'tls', 'protocol'] = 'DoT'
measurement_successful_heatmap.loc[measurement_successful_heatmap.protocol == 'tcp', 'protocol'] = 'DoTCP'
measurement_successful_heatmap.loc[measurement_successful_heatmap.protocol == 'udp', 'protocol'] = 'DoUDP'

measurement_successful_heatmap.loc[measurement_successful_heatmap.vp == 'AF-SOUTH-1', 'vp'] = 'AF'
measurement_successful_heatmap.loc[measurement_successful_heatmap.vp == 'AP-NORTHEAST-3', 'vp'] = 'AS'
measurement_successful_heatmap.loc[measurement_successful_heatmap.vp == 'AP-SOUTHEAST-3', 'vp'] = 'OC'
measurement_successful_heatmap.loc[measurement_successful_heatmap.vp == 'EU-CENTRAL-1', 'vp'] = 'EU'
measurement_successful_heatmap.loc[measurement_successful_heatmap.vp == 'SA-EAST-1', 'vp'] = 'SA'
#measurement_successful_heatmap.loc[measurement_successful_heatmap.vp == 'US-EAST-1', 'vp'] = 'NA-E'
measurement_successful_heatmap.loc[measurement_successful_heatmap.vp == 'US-WEST-1', 'vp'] = 'NA'

measurement_successful_heatmap

In [ ]:
def sort_heatmap(time_heatmap):
    # sort by number of resolvers
    # \ac{EU} with 130 resolvers, followed by \ac{AS} with 128, \ac{NA} with 49, and \ac{AF}, \ac{OC}, and \ac{SA} with 2 resolvers each.

    time_heatmap_tmp = time_heatmap[time_heatmap.index == 'EU']
    time_heatmap = time_heatmap[time_heatmap.index != 'EU']
    time_heatmap = time_heatmap.append(time_heatmap_tmp)

    time_heatmap_tmp = time_heatmap[time_heatmap.index == 'AS']
    time_heatmap = time_heatmap[time_heatmap.index != 'AS']
    time_heatmap = time_heatmap.append(time_heatmap_tmp)

    time_heatmap_tmp = time_heatmap[time_heatmap.index == 'NA']
    time_heatmap = time_heatmap[time_heatmap.index != 'NA']
    time_heatmap = time_heatmap.append(time_heatmap_tmp)

    time_heatmap_tmp = time_heatmap[time_heatmap.index == 'AF']
    time_heatmap = time_heatmap[time_heatmap.index != 'AF']
    time_heatmap = time_heatmap.append(time_heatmap_tmp)

    time_heatmap_tmp = time_heatmap[time_heatmap.index == 'OC']
    time_heatmap = time_heatmap[time_heatmap.index != 'OC']
    time_heatmap = time_heatmap.append(time_heatmap_tmp)

    time_heatmap_tmp = time_heatmap[time_heatmap.index == 'SA']
    time_heatmap = time_heatmap[time_heatmap.index != 'SA']
    time_heatmap = time_heatmap.append(time_heatmap_tmp)
    
    return time_heatmap

### resolve time

In [ ]:
query_time_total = measurement_successful_heatmap.groupby('protocol', as_index=False)['query_time'].median()
query_time_total['vp'] = 'Total'
query_time_total = query_time_total[['protocol', 'vp', 'query_time']]

query_time_heatmap = measurement_successful_heatmap.groupby(['protocol', 'vp'], as_index=False
                               )['query_time'].median().append(query_time_total)
query_time_heatmap['query_time'] = query_time_heatmap['query_time'] / 1000000
query_time_heatmap = pd.crosstab(query_time_heatmap['vp'], query_time_heatmap['protocol'],
                          values=query_time_heatmap['query_time'], aggfunc='median'
                        )

# move Total to the top
query_time_heatmap_total = query_time_heatmap[query_time_heatmap.index == 'Total']
query_time_heatmap_tmp = query_time_heatmap[query_time_heatmap.index != 'Total']
query_time_heatmap = query_time_heatmap_total.append(query_time_heatmap_tmp)
query_time_heatmap = query_time_heatmap[['DoQ', 'DoUDP', 'DoTCP', 'DoT', 'DoH']]
query_time_heatmap = query_time_heatmap[['DoUDP', 'DoTCP', 'DoQ', 'DoH', 'DoT']]
query_time_heatmap = sort_heatmap(query_time_heatmap)
query_time_heatmap

### handshake time

In [ ]:
handshake_time_total = measurement_successful_heatmap.groupby('protocol', as_index=False)['handshake_time'].median()
handshake_time_total['vp'] = 'Total'
handshake_time_total = handshake_time_total[['protocol', 'vp', 'handshake_time']]

handshake_time_heatmap = measurement_successful_heatmap.groupby(['protocol', 'vp'], as_index=False
                               )['handshake_time'].median().append(handshake_time_total)
handshake_time_heatmap = handshake_time_heatmap.fillna(0)
handshake_time_heatmap['handshake_time'] = handshake_time_heatmap['handshake_time'] / 1000000
handshake_time_heatmap = pd.crosstab(handshake_time_heatmap['vp'], handshake_time_heatmap['protocol'],
                          values=handshake_time_heatmap['handshake_time'], aggfunc='median'
                        )

# move Total to the top
handshake_time_heatmap_total = handshake_time_heatmap[handshake_time_heatmap.index == 'Total']
handshake_time_heatmap_tmp = handshake_time_heatmap[handshake_time_heatmap.index != 'Total']
handshake_time_heatmap = handshake_time_heatmap_total.append(handshake_time_heatmap_tmp)
handshake_time_heatmap = handshake_time_heatmap[['DoQ', 'DoUDP', 'DoTCP', 'DoT', 'DoH']]
handshake_time_heatmap['DoUDP'] = np.NaN
handshake_time_heatmap = handshake_time_heatmap[['DoUDP', 'DoTCP', 'DoQ', 'DoH', 'DoT']]
handshake_time_heatmap = sort_heatmap(handshake_time_heatmap)
handshake_time_heatmap


### plot

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 3), gridspec_kw = {'wspace':0.07, 'hspace':0.1, 'width_ratios': [5, 6]}, sharex=True)

ax1 = sns.heatmap(handshake_time_heatmap,
            annot=True,
            cmap='RdYlGn_r', #'RdYlBu_r',  # "RdYlGn_r",
            #cbar_kws={'label' : 'Handshake Time [ms]'},
            ax=ax1,
            fmt='.1f',
            vmin=0,
            vmax=400,
            xticklabels=True,
            linewidths=3,
            cbar=False
           )
ax1.set_xlabel('')
ax1.set_ylabel('')
ax1.tick_params(left=False, bottom=False)
ax1.set_facecolor('.95')

ax2 = sns.heatmap(query_time_heatmap,
            annot=True,
            cmap='RdYlGn_r', #'RdYlBu_r',  # "RdYlGn_r",
            #cbar_kws={'label' : 'Resolve Time [ms]'},
            ax=ax2,
            fmt='.1f',
            vmin=0,
            vmax=400,
            xticklabels=True,
            yticklabels=False,
            linewidths=3
           )
ax2.set_xlabel('')
ax2.set_ylabel('')
ax2.tick_params(left=False, bottom=False)

fig.show()
fig.savefig('handshake-resolve-heatmap.pdf', bbox_inches='tight')